In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformer_lens import HookedTransformer 
import torch
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from jaxtyping import Float
import tqdm
import transformer_lens.utils as utils
from functools import partial
from tabulate import tabulate

In [2]:
#Loading model - let's use Pythia 
model = HookedTransformer.from_pretrained("EleutherAI/pythia-410m-deduped")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m-deduped")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-410m-deduped into HookedTransformer


In [3]:
tokens = tokenizer.encode("new simple network architecture, the Transformer", return_tensors="pt")
with torch.no_grad():
    logits, clean_cache = model.run_with_cache(tokens)

In [4]:
#lens_output = lenses.forward(hidden_states_at_last_token[:, layer_index, :], layer_index)

class NewLens:
    def __init__(self, model):
        self._model = model

    def forward(self, cache_to_patch, layer_index):
        #cache_to_patch : batch x d_model
        n_batch = cache_to_patch.shape[0]

        def resid_pre_hook(
            activations: torch.Tensor,
            hook: HookPoint,
        ):
            return cache_to_patch
        
        resid_pre_hook_fn = partial(resid_pre_hook)
        self._model.reset_hooks(including_permanent=True)
        self._model.blocks[layer_index].hook_resid_pre.add_hook(resid_pre_hook_fn)
        fake_inputs = torch.zeros(n_batch, 1, device="cpu").int() #patching with batch x position long 1
        out = self._model(fake_inputs)
        return out

In [15]:
#lens_output = lenses.forward(hidden_states_at_last_token[:, layer_index, :], layer_index)

class NewLens:
    def __init__(self, model):
        self._model = model

    def forward(self, cache_to_patch, layer_index):
        #cache_to_patch : batch x d_model
        n_batch = cache_to_patch.shape[0]

        def hook(
            model,input,output
        ):
            return cache_to_patch
        
        handle = self._model.gpt_neox.layers[1].register_forward_hook(hook)
        fake_inputs = torch.zeros(n_batch, 1, device="cpu").int() #patching with batch x position long 1
        out = self._model(fake_inputs)
        handle.remove()

        return out

In [5]:
lens = NewLens(model)
n_pos = clean_cache[utils.get_act_name("resid_pre", 0)].shape[1]
n_batch = clean_cache[utils.get_act_name("resid_pre", 0)].shape[0]
n_topk = 1

patching_result = torch.zeros((n_batch, model.cfg.n_layers, n_pos, n_topk), device="cpu") 

for layer in range(model.cfg.n_layers):
    for position in range(n_pos):
        lens_output = lens.forward(clean_cache[utils.get_act_name("resid_pre", layer)][:, position, :].unsqueeze(0), layer)
        _ , patching_result[:, layer, position, :] = torch.topk(lens_output.squeeze(), k=n_topk)

In [4]:
def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    # Each HookPoint has a name attribute giving the name of the hook.
    print(hook.name)
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, 0, :] = clean_resid_pre[:, position, :]
    return resid_pre

# We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
num_positions = len(tokens[0])
patching_result = torch.zeros((model.cfg.n_layers, num_positions, 1), device=model.cfg.device)

for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    for position in range(num_positions):
        print(position)
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = model.run_with_hooks(torch.tensor([[0]], device='cuda:0'), fwd_hooks=[
            (utils.get_act_name("resid_post", layer), temp_hook_fn)
        ])
        # Calculate the logit difference      
        _, patching_result[layer, position] = torch.topk(patched_logits.squeeze(), k=1)

  0%|          | 0/24 [00:00<?, ?it/s]

0
blocks.0.hook_resid_post
1
blocks.0.hook_resid_post
2
blocks.0.hook_resid_post
3
blocks.0.hook_resid_post
4
blocks.0.hook_resid_post
5
blocks.0.hook_resid_post


  4%|▍         | 1/24 [00:00<00:21,  1.09it/s]

6
blocks.0.hook_resid_post
7
blocks.0.hook_resid_post
0
blocks.1.hook_resid_post
1
blocks.1.hook_resid_post
2
blocks.1.hook_resid_post
3
blocks.1.hook_resid_post
4
blocks.1.hook_resid_post
5
blocks.1.hook_resid_post
6
blocks.1.hook_resid_post
7
blocks.1.hook_resid_post


  8%|▊         | 2/24 [00:01<00:18,  1.16it/s]

...

0
blocks.22.hook_resid_post
1
blocks.22.hook_resid_post
2
blocks.22.hook_resid_post
3
blocks.22.hook_resid_post
4
blocks.22.hook_resid_post
5
blocks.22.hook_resid_post
6
blocks.22.hook_resid_post
7


 96%|█████████▌| 23/24 [00:19<00:00,  1.18it/s]

blocks.22.hook_resid_post
0
blocks.23.hook_resid_post
1
blocks.23.hook_resid_post
2
blocks.23.hook_resid_post
3
blocks.23.hook_resid_post
4
blocks.23.hook_resid_post
5
blocks.23.hook_resid_post
6
blocks.23.hook_resid_post
7


100%|██████████| 24/24 [00:20<00:00,  1.20it/s]

blocks.23.hook_resid_post


In [6]:
patching_result = patching_result.squeeze().unsqueeze(-1)
patching_result.shape

torch.Size([24, 8, 1])

In [9]:
patching_result.shape

torch.Size([24, 8, 1])

In [10]:
patching_result

tensor([[[6.4000e+01],
         [1.3000e+01],
         [1.3000e+01],
         [1.8700e+02],
         [1.4070e+03],
         [1.8700e+02],
         [1.4000e+01],
         [1.5000e+01]],

        [[6.4000e+01],
         [1.3000e+01],
         [1.8700e+02],
         [1.5000e+01],
         [2.8500e+02],
         [1.0000e+01],
         [4.4800e+03],
         [1.4444e+04]],

        [[6.4000e+01],
         [2.9690e+03],
         [2.9900e+03],
         [1.0336e+04],
         [2.8500e+02],
         [1.0000e+01],
         [4.4800e+03],
         [1.4444e+04]],

        [[6.4000e+01],
         [2.9690e+03],
         [2.9900e+03],
         [1.0336e+04],
         [2.8500e+02],
         [2.5300e+02],
         [4.4800e+03],
         [6.1400e+02]],

        [[6.4000e+01],
         [2.9690e+03],
         [2.9900e+03],
         [1.0336e+04],
         [1.8700e+02],
         [2.5300e+02],
         [4.4800e+03],
         [8.6950e+03]],

        [[6.4000e+01],
         [2.9690e+03],
         [2.9900e+03],
 

In [5]:
def stringify(x):
    return model.to_string(x)
result = [[[stringify(top) for top in element]for element in subarray] for subarray in patching_result.int()]
strtoks = [stringify(tok) for tok in tokens[0]]

In [6]:
_ , clean_predicts = torch.topk(logits.squeeze(), k=5)
clean_predicts_text = [[stringify(tok) for tok in elem] for elem in clean_predicts.int()]

In [7]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════╤═════════════╤══════════════╤═══════════════════╤════════════╤══════════════╤════════════╤══════════════════╕
│ new   │  simple     │  network     │  architecture     │ ,          │  the         │  Trans     │ former           │
╞═══════╪═════════════╪══════════════╪═══════════════════╪════════════╪══════════════╪════════════╪══════════════════╡
│ ['_'] │ [',']       │ [',']        │ ['\n']            │ [' ed']    │ ['\n']       │ ['-']      │ ['.']            │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼────────────┼──────────────────┤
│ ['_'] │ [',']       │ ['\n']       │ ['.']             │ [' and']   │ [')']        │ [' Trans'] │ ['Tree']         │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼────────────┼──────────────────┤
│ ['_'] │ [' simple'] │ [' network'] │ [' architecture'] │ [' and']   │ [')']        │ [' Trans'] │ ['Tree']         │
├───────┼─────────────┼──────────────┼──────────

In [9]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════╤═════════════╤══════════════╤═══════════════════╤════════════╤══════════════╤═════════════╤══════════════════╕
│ new   │  simple     │  network     │  architecture     │ ,          │  the         │  Trans      │ former           │
╞═══════╪═════════════╪══════════════╪═══════════════════╪════════════╪══════════════╪═════════════╪══════════════════╡
│ ['_'] │ [' and']    │ [',']        │ [',']             │ [' and']   │ [' same']    │ ['actions'] │ ['.']            │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ ['_'] │ [',']       │ [',']        │ ['\n']            │ [' ed']    │ ['\n']       │ ['-']       │ ['.']            │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ ['_'] │ [',']       │ ['\n']       │ ['.']             │ [' and']   │ [')']        │ [' Trans']  │ ['Tree']         │
├───────┼─────────────┼──────────────┼──

In [7]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════╤═════════════╤══════════════╤═══════════════════╤════════════╤══════════════╤═════════════╤══════════════════╕
│ new   │  simple     │  network     │  architecture     │ ,          │  the         │  Trans      │ former           │
╞═══════╪═════════════╪══════════════╪═══════════════════╪════════════╪══════════════╪═════════════╪══════════════════╡
│ ['_'] │ [' and']    │ [',']        │ [',']             │ [' and']   │ [' same']    │ ['actions'] │ ['.']            │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ ['_'] │ [',']       │ [',']        │ ['\n']            │ [' ed']    │ ['\n']       │ ['-']       │ ['.']            │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ ['_'] │ [',']       │ ['\n']       │ ['.']             │ [' and']   │ [')']        │ [' Trans']  │ ['Tree']         │
├───────┼─────────────┼──────────────┼──

In [67]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════════════════╤═════════════╤═══════════════╤═══════════════════╤════════════╤═════════════════╤════════════╤═══════════════════╕
│ new               │  simple     │  network      │  architecture     │ ,          │  the            │  Trans     │ former            │
╞═══════════════════╪═════════════╪═══════════════╪═══════════════════╪════════════╪═════════════════╪════════════╪═══════════════════╡
│ [' and']          │ ['?']       │ ['*']         │ ['\n']            │ ['\n']     │ [' s']          │ ['-']      │ ['\n']            │
├───────────────────┼─────────────┼───────────────┼───────────────────┼────────────┼─────────────────┼────────────┼───────────────────┤
│ ['1']             │ [' is']     │ ['\n']        │ ['\n']            │ [' and']   │ [' the']        │ [' Trans'] │ ['{']             │
├───────────────────┼─────────────┼───────────────┼───────────────────┼────────────┼─────────────────┼────────────┼───────────────────┤
│ ['sw']            │ [' and']    │ [' network']

In [17]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════╤═════════════╤══════════════╤═══════════════════╤════════════╤══════════════╤═════════════╤══════════════════╕
│ new   │  simple     │  network     │  architecture     │ ,          │  the         │  Trans      │ former           │
╞═══════╪═════════════╪══════════════╪═══════════════════╪════════════╪══════════════╪═════════════╪══════════════════╡
│ ['_'] │ [' and']    │ [',']        │ [',']             │ [' and']   │ [' same']    │ ['actions'] │ ['.']            │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ ['_'] │ [',']       │ [',']        │ ['\n']            │ [' ed']    │ ['\n']       │ ['-']       │ ['.']            │
├───────┼─────────────┼──────────────┼───────────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ ['_'] │ [',']       │ ['\n']       │ ['.']             │ [' and']   │ [')']        │ [' Trans']  │ ['Tree']         │
├───────┼─────────────┼──────────────┼──

In [30]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════════════════════════════════════╤═════════════════════════════════════════════╤════════════════════════════════════════════╤═════════════════════════════════════════════════╤═════════════════════════════════════════════╤═══════════════════════════════════════════════════╤══════════════════════════════════════════════════════════╤════════════════════════════════════════════════╤═══════════════════════════════════════════╤══════════════════════════════════════════════════╤═════════════════════════════════════════════════╤═══════════════════════════════════════════════════════╤══════════════════════════════════════════════╤═════════════════════════════════════════════════╤══════════════════════════════════════════════════════╤═══════════════════════════════════════════════╕
│ When                                  │  John                                       │  and                                       │  Mary                                           │  went                      

In [23]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════════════════════════════════════════════════════╤══════════════════════════════════════════════════════╤════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════╤════════════════════════════════════════════════════════╤═════════════════════════════════════════════════╤════════════════════════════════════════════════════════╕
│ New                                                   │  simple                                              │  network                                               │  architecture                                                                      │ ,                                         │  the                                                   │  Trans                                          │ former                                                 │
╞═══════════════════════════════════════════════════════╪═════════════════

In [14]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒══════════════════════════════════════════╤══════════════════════════════════════════╤══════════════════════════════════════════════╤═══════════════════════════════════════════════╤═══════════════════════════════════════════════╤═══════════════════════════════════════╤════════════════════════════════════════════════════════════╤═════════════════════════════════════════╤═════════════════════════════════════════════════╤═══════════════════════════════════════════════╤═════════════════════════════════════════════════════╤═══════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════╕
│ How                                      │  can                                     │  I                                           │  kill                                         │  myself                                       │ ?                                     │  Answer                                                

## Comparing TunedLens and ourLens over IOI task

In [11]:
import torch
from tuned_lens.nn.lenses import TunedLens, LogitLens
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device('cpu')
# To try a diffrent modle / lens check if the lens is avalible then modify this code
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tuned_lens = TunedLens.from_model_and_pretrained(model)
tuned_lens = tuned_lens.to(device)
logit_lens = LogitLens.from_model(model)

In [12]:
from tuned_lens.plotting import PredictionTrajectory
import ipywidgets as widgets
from plotly import graph_objects as go


def make_plot(lens, text, layer_stride, statistic, token_range):
    input_ids = tokenizer.encode(text)
    targets = input_ids[1:] + [tokenizer.eos_token_id]

    if len(input_ids) == 0:
        return widgets.Text("Please enter some text.")
    
    if (token_range[0] == token_range[1]):
        return widgets.Text("Please provide valid token range.")
    pred_traj = PredictionTrajectory.from_lens_and_model(
        lens=lens,
        model=model,
        input_ids=input_ids,
        tokenizer=tokenizer,
        targets=targets,
    ).slice_sequence(slice(*token_range))

    return getattr(pred_traj, statistic)().stride(layer_stride).figure(
        title=f"{lens.__class__.__name__} ({model.name_or_path}) {statistic}",
    )

style = {'description_width': 'initial'}
statistic_wdg = widgets.Dropdown(
    options=[
        ('Entropy', 'entropy'),
        ('Cross Entropy', 'cross_entropy'),
        ('Forward KL', 'forward_kl'),
    ],
    description='Select Statistic:',
    style=style,
)
text_wdg = widgets.Textarea(
    description="Input Text",
    value="it was the best of times, it was the worst of times",
)
lens_wdg = widgets.Dropdown(
    options=[('Tuned Lens', tuned_lens), ('Logit Lens', logit_lens)],
    description='Select Lens:',
    style=style,
)

layer_stride_wdg = widgets.BoundedIntText(
    value=2,
    min=1,
    max=10,
    step=1,
    description='Layer Stride:',
    disabled=False
)

token_range_wdg = widgets.IntRangeSlider(
    description='Token Range',
    min=0,
    max=1,
    step=1,
    style=style,
)


def update_token_range(*args):
    token_range_wdg.max = len(tokenizer.encode(text_wdg.value))

update_token_range()

token_range_wdg.value = [0, token_range_wdg.max]
text_wdg.observe(update_token_range, 'value')

interact = widgets.interact.options(manual_name='Run Lens', manual=True)

plot = interact(
    make_plot,
    text=text_wdg,
    statistic=statistic_wdg,
    lens=lens_wdg,
    layer_stride=layer_stride_wdg,
    token_range=token_range_wdg,
)

interactive(children=(Dropdown(description='Select Lens:', options=(('Tuned Lens', TunedLens(
  (unembed): Une…

In [2]:
hooked_model = HookedTransformer.from_pretrained("gpt2")

Loaded pretrained model gpt2 into HookedTransformer


In [4]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [5]:
tokens = tokenizer.encode("Jingle bells jingle bells jingle all the", return_tensors="pt")
with torch.no_grad():
    logits, clean_cache = hooked_model.run_with_cache(tokens)

In [6]:
def residual_stream_patching_hook(
    resid_post: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    clean_resid_post = clean_cache[hook.name]
    resid_post[:, 0, :] = clean_resid_post[:, position, :]
    return resid_post

num_positions = len(tokens[0])
patching_result = torch.zeros((hooked_model.cfg.n_layers, num_positions, 50), device=hooked_model.cfg.device)

for layer in tqdm.tqdm(range(hooked_model.cfg.n_layers)):
    for position in range(num_positions):
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = hooked_model.run_with_hooks(torch.tensor([[0]], device='cuda:0'), fwd_hooks=[
            (utils.get_act_name("resid_post", layer), temp_hook_fn)
        ])
        # Calculate the logit difference      
        _, patching_result[layer, position] = torch.topk(patched_logits.squeeze(), k=50)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:09<00:00,  1.31it/s]


In [7]:
def stringify(x):
    return hooked_model.to_string(x)
result = [[[stringify(top) for top in element]for element in subarray] for subarray in patching_result.int()]
strtoks = [stringify(tok) for tok in tokens[0]]

In [8]:
_ , clean_predicts = torch.topk(logits.squeeze(), k=5)
clean_predicts_text = [[stringify(tok) for tok in elem] for elem in clean_predicts.int()]
result.append(clean_predicts_text)

In [9]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [94]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

see how there's when John and Mary went to church which is due to the association in the embeddings (I think)

In [88]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

generations on hook_resid_pre

In [83]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [78]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

layer 8: first layer where there's Mary in the top 50 but after John, layer 9 molto più avanti
In the case in which you would need to have John predicted there's not John in the top 50 of layer 8 but is present in layer 9


In [65]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒═══════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════╕
│ The                                                                   │  Tour                  

In [55]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════

In [50]:
print(tabulate(result, headers=strtoks, tablefmt="fancy_grid"))

╒════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════

Trying Llama2

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                            torch_dtype="auto", load_in_4bit=True,
                                             low_cpu_mem_usage=True)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          torch_dtype="auto")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

C:\Users\nicol\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nicol\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the f

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 